# Model Preparation
In this notebook, we will determine the best pre-trained model (on COCO dataset) for our use-case, conduct a few-shot eager learning and save the model to be used by the webapp.

> **Goal**: Determine the best pre-trained object detection model to detect objects and their shapes.

## Data Flow
1. An image will be recieved from the deployed webpage.
2. Image will be pre-processed and converted into a numpy array and to a tensor.
3. Model will make the prediction based on the image provided.
4. Post-processing will convert the model's prediction into statistics that will be displayed to the user.

Notes: The model will predict the object and create a bounding box around around each object in the image. In post-processing, the shape of each object class is pre-defined. The webpage will be returned the processed image (with bounding boxes), list of objects in the image, along with their shapes.


# 1. Determine the Ideal Model
In this section, we will use the [TensorFlow 2 Detection Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md) table and metrics to determine the ideal model for the use-case. As our goal is to deploy the model on a webpage, we would want to ensure the right balance between speed and accuracy (mAP).

In [2]:
# import modules
import pandas as pd
import tensorflow as tf
import pathlib

In [4]:
# Load data about pretrained model
df = pd.read_csv('TF2-detection-model-zoo-meta.csv')
df.head()

,Model name,Speed (ms),COCO mAP,Outputs
0,CenterNet HourGlass104 512x512,70,41.9,Boxes
1,CenterNet HourGlass104 Keypoints 512x512,76,50.7,Boxes/Keypoints
2,CenterNet HourGlass104 1024x1024,197,44.5,Boxes
3,CenterNet HourGlass104 Keypoints 1024x1024,211,53.6,Boxes/Keypoints
4,CenterNet Resnet50 V1 FPN 512x512,27,31.2,Boxes


We want the model with the highest accuracy and lowest inference time (fastest speed). Therefore, we can create a new variables that takes the ratio of the two metrics - `COCO mAP / Speed (ms)`. Maximizing this ratio will give us the models that will work best for the use-case. 
Additionally, mimimum and maximum values for speed and accruacy are defined as below:
- min mAP: 35.0
- max Speed: 50ms

Note: Many assumptions are in the above model selection process. One of them being the model size - this is usaully given in terms of `training parameters`. As these models are already trained, the size of the models is not taken into consideration. However, many other factors that could impact the decision of selecting the perfect model are assumed to have negligible impact. 

In [5]:
df['mAP/Speed Ratio'] = df['COCO mAP'] / df['Speed (ms)']
df.sort_values(by='mAP/Speed Ratio', ascending=False).head(10)

,Model name,Speed (ms),COCO mAP,Outputs,mAP/Speed Ratio
10,CenterNet MobileNetV2 FPN Keypoints 512x512,6,41.7,Keypoints,6.950000
9,CenterNet MobileNetV2 FPN 512x512,6,23.4,Boxes,3.900000
5,CenterNet Resnet50 V1 FPN Keypoints 512x512,30,40.0,Boxes/Keypoints,1.333333
8,CenterNet Resnet50 V2 Keypoints 512x512,30,37.9,Boxes/Keypoints,1.263333
4,CenterNet Resnet50 V1 FPN 512x512,27,31.2,Boxes,1.155556
7,CenterNet Resnet50 V2 512x512,27,29.5,Boxes,1.092593
19,SSD MobileNet v2 320x320,19,20.2,Boxes,1.063158
21,SSD MobileNet V2 FPNLite 320x320,22,22.2,Boxes,1.009091
6,CenterNet Resnet101 V1 FPN 512x512,34,34.2,Boxes,1.005882
11,EfficientDet D0 512x512,39,33.6,Boxes,0.861538


Consideirng the ratio and limits of accuracy and speed, the best pre-trained model to be used for the use-case is **`CenterNet Resnet50 V1 FPN 512x512`**

# 2. Loading Pre-trained Model
In this section, we will be loading the model and testing it's performance on random images.

In [7]:
# import modules
import matplotlib
import matplotlib.pyplot as plt

import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

%matplotlib inline

ModuleNotFoundError: No module named 'object_detection'

In [6]:
# path to the downloaded pre-trained model.
path = pathlib.Path("../models/centernet_resnet101_v1_fpn_512x512_coco17_tpu-8/saved_model")
model = None

if path.exists():
    model = tf.saved_model.load(str(path))
    
model

Metal device set to: Apple M1


2022-04-13 19:05:16.152598: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-13 19:05:16.152898: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x2f1fe3be0>